In [ ]:
# If you would like to build from scratch. 

from modules.exploration import explore_data, show_image    # For EDA, not needed
from modules.preprocess import get_data, stratified_split, process_all    # For preprocessing (Already done, can just call from data/preprocessed)
from modules.datasets import Covid19DataSet    # (Already done, can just call from data/preprocessed)
from modules.models import get_model    # Create a fresh model
from modules.training import train_model, get_metrics    # For training and testing model
from modules.metrics import plot_roc_auc    # , create_table, plot_loss_and_metric